In [38]:
import pandas as pd
import numpy as np
import sys
import os
from os import system
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB

In [2]:
data_dir = "data"
for root, dirs, files in os.walk(data_dir, topdown=False):
    for name in files:
        print(os.path.join(root, name))
    for name in dirs:
        print(os.path.join(root, name))

data/title.basics.tsv/data.tsv
data/data-sarc-sample/sarc/sarcastic_879.txt
data/data-sarc-sample/sarc/sarcastic_223.txt
data/data-sarc-sample/sarc/sarcastic_318.txt
data/data-sarc-sample/sarc/sarcastic_118.txt
data/data-sarc-sample/sarc/sarcastic_932.txt
data/data-sarc-sample/sarc/sarcastic_129.txt
data/data-sarc-sample/sarc/sarcastic_494.txt
data/data-sarc-sample/sarc/sarcastic_662.txt
data/data-sarc-sample/sarc/sarcastic_592.txt
data/data-sarc-sample/sarc/sarcastic_135.txt
data/data-sarc-sample/sarc/sarcastic_470.txt
data/data-sarc-sample/sarc/sarcastic_87.txt
data/data-sarc-sample/sarc/sarcastic_209.txt
data/data-sarc-sample/sarc/sarcastic_869.txt
data/data-sarc-sample/sarc/sarcastic_743.txt
data/data-sarc-sample/sarc/sarcastic_952.txt
data/data-sarc-sample/sarc/sarcastic_933.txt
data/data-sarc-sample/sarc/sarcastic_904.txt
data/data-sarc-sample/sarc/sarcastic_230.txt
data/data-sarc-sample/sarc/sarcastic_784.txt
data/data-sarc-sample/sarc/sarcastic_915.txt
data/data-sarc-sample/sar

In [3]:
# Combine to TSV
if 'data' not in os.getcwd(): 
    os.chdir("data")
system("pwd")
if not os.path.exists('combined.csv'):
    system("cat *.csv > combined.csv")
os.chdir("../")

/home/pavle/Documents/Git/medvidov/dsci550/data


In [4]:
pix_df = pd.read_csv("data/combined.csv")
pix_df.sort_values(by=['Account Created Date'])
pix_df

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest
0,105133,STY1654473727,8255,USR1628503147,others,13,2. Looking back : On the bench once more,On the Bench Once More continues....\n\nWhile ...,https://image.pixstory.com/Pixstory-image-1628...,2021-08-09 15:29:07,"looking back, books"
1,105132,STY1654473498,8601,USR1630424770,others,13,3. Contemporary Afghanistan : The 'Bonn Agreem...,International Community in\nAfghanistan contin...,https://image.pixstory.com/Pixstory-image-1630...,2021-08-31 21:16:10,"contemporary afghanistan, books"
2,105131,STY1654473332,28647,USR1646131566,NaN,12,20. A Question of Pride : The 'Chushi Gangdrug...,Enter the Dragon: Chinese Invasion \nof Tibet ...,https://image.pixstory.com/Pixstory-image-1646...,2022-03-01 16:16:06,"a question of pride, books"
3,105130,STY1654472717,40917,USR1654101217,male,23,From Undesirable to Undeniable - The Cody Rhod...,WRESTLING HAS MORE THAN ONE ROYAL FAMILY!! \n\...,https://image.pixstory.com/Pixstory-image-1654...,2022-06-01 22:03:37,"cody rhodes, wwe, seth rollins, aew"
4,105129,STY1654472629,30966,USR1649551870,NaN,12,Aboriginal elders voice concerns over proposed...,It is far too soon to be talking about a propo...,https://image.pixstory.com/Pixstory-image-1654...,2022-04-10 06:21:10,"aboriginal, indigenous people, indigenous righ..."
...,...,...,...,...,...,...,...,...,...,...,...
95001,130728,STY1661176231,77742,USR1661080107,NaN,19,Lord,"<p dir=""ltr"">Krishna</p>",https://image.pixstory.com/Pixstory-image-1661...,2022-08-21 16:38:27,"Technology, History, Food, Entertainment, Spor..."
95002,130727,STY1661176187,78675,USR1661176067,NaN,18,College,"<p dir=""ltr"">College </p>",https://image.pixstory.com/Pixstory-image-1661...,2022-08-22 19:17:47,"Technology, History, Food, Entertainment, Spor..."
95003,130726,STY1661176171,78674,USR1661176023,NaN,18,🇮🇳,"<p dir=""ltr"">&#127470;&#127475;</p>",https://image.pixstory.com/Pixstory-image-1661...,2022-08-22 19:17:03,"Health, Politics, Economy, Climate change , Te..."
95004,130725,STY1661175883,78668,USR1661175804,NaN,22,#Gym,"<p dir=""ltr"">#Gym</p>",https://image.pixstory.com/Pixstory-image-1661...,2022-08-22 19:13:24,"Technology, History, Food, Entertainment, Spor..."


In [5]:
# Sarcasm classification using corpus V2
# https://thecleverprogrammer.com/2021/08/24/sarcasm-detection-with-machine-learning/

# Create sarcasm data directory
sarc_data_dir = "data/sarcasm_v2/"

# Placeholder df
sarc_df = pd.DataFrame()

# Get all types of sarcasm into the df
for root, dirs, files in os.walk(sarc_data_dir, topdown=False):
    for name in files:
        df = pd.read_csv(os.path.join(root, name))
        sarc_df = pd.concat([sarc_df, df])

# Group by the class and make it easier for use
sarc_df.groupby('class')
sarc_df = sarc_df.replace('notsarc',0)
sarc_df = sarc_df.replace('sarc',1)
sarc_df

,class,id,text
0,0,1,"Archie, the ONLY issue that gays don't have a ..."
1,0,2,"No, not really. All that is different is the n..."
2,0,3,It's ashame that everyone keeps looking for th...
3,0,4,"Almost? Usually, that is true, and it involves..."
4,0,5,And so have animals. Plants have been wiped ou...
...,...,...,...
6515,1,6516,depends on when the baby bird died. run alon...
6516,1,6517,"ok, sheesh, to clarify, women who arent aborti..."
6517,1,6518,so.. eh?? hows this sound? will it fly w...
6518,1,6519,"I think we should put to a vote, the right of ..."


In [40]:
# Define X and Y
x = np.array(sarc_df["text"])
y = np.array(sarc_df["class"])

# Count vectorizer for sarcasm detection
cv = CountVectorizer()

# Fit and transform the data and make stratified train and test sets
X = cv.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify = y)

# Classify and score on the data. Using ComplementNB because it sometimes outperforms MultinomialNB
# bern = BernoulliNB()
# bern.fit(X_train, y_train)
# mult = MultinomialNB()
# mult.fit(X_train, y_train)
comp = ComplementNB()
comp.fit(X_train, y_train)
# print(bern.score(X_test, y_test))
# print(mult.score(X_test, y_test))
# print(comp.score(X_test, y_test))

ComplementNB()

In [43]:
# Create new column to append
sarcasm = []

# Detect sarcasm
for i in pix_df.iterrows():
    text = [np.str_(i[1]['Narrative'])]
    data = cv.transform(text)
    output = comp.predict(data)
    sarcasm.append(output[0])

# Add sarcasm column to df
pix_df['Sarcasm'] = sarcasm

In [44]:
pix_df

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest,Sarcasm
0,105133,STY1654473727,8255,USR1628503147,others,13,2. Looking back : On the bench once more,On the Bench Once More continues....\n\nWhile ...,https://image.pixstory.com/Pixstory-image-1628...,2021-08-09 15:29:07,"looking back, books",0
1,105132,STY1654473498,8601,USR1630424770,others,13,3. Contemporary Afghanistan : The 'Bonn Agreem...,International Community in\nAfghanistan contin...,https://image.pixstory.com/Pixstory-image-1630...,2021-08-31 21:16:10,"contemporary afghanistan, books",0
2,105131,STY1654473332,28647,USR1646131566,NaN,12,20. A Question of Pride : The 'Chushi Gangdrug...,Enter the Dragon: Chinese Invasion \nof Tibet ...,https://image.pixstory.com/Pixstory-image-1646...,2022-03-01 16:16:06,"a question of pride, books",0
3,105130,STY1654472717,40917,USR1654101217,male,23,From Undesirable to Undeniable - The Cody Rhod...,WRESTLING HAS MORE THAN ONE ROYAL FAMILY!! \n\...,https://image.pixstory.com/Pixstory-image-1654...,2022-06-01 22:03:37,"cody rhodes, wwe, seth rollins, aew",0
4,105129,STY1654472629,30966,USR1649551870,NaN,12,Aboriginal elders voice concerns over proposed...,It is far too soon to be talking about a propo...,https://image.pixstory.com/Pixstory-image-1654...,2022-04-10 06:21:10,"aboriginal, indigenous people, indigenous righ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
95001,130728,STY1661176231,77742,USR1661080107,NaN,19,Lord,"<p dir=""ltr"">Krishna</p>",https://image.pixstory.com/Pixstory-image-1661...,2022-08-21 16:38:27,"Technology, History, Food, Entertainment, Spor...",1
95002,130727,STY1661176187,78675,USR1661176067,NaN,18,College,"<p dir=""ltr"">College </p>",https://image.pixstory.com/Pixstory-image-1661...,2022-08-22 19:17:47,"Technology, History, Food, Entertainment, Spor...",1
95003,130726,STY1661176171,78674,USR1661176023,NaN,18,🇮🇳,"<p dir=""ltr"">&#127470;&#127475;</p>",https://image.pixstory.com/Pixstory-image-1661...,2022-08-22 19:17:03,"Health, Politics, Economy, Climate change , Te...",0
95004,130725,STY1661175883,78668,USR1661175804,NaN,22,#Gym,"<p dir=""ltr"">#Gym</p>",https://image.pixstory.com/Pixstory-image-1661...,2022-08-22 19:13:24,"Technology, History, Food, Entertainment, Spor...",1


In [16]:
title_data = "data/title.basics.tsv/data.tsv"
title_df = pd.read_csv(title_data, sep="\t")
title_df = title_df.drop(labels = ['originalTitle','runtimeMinutes', 'isAdult','endYear'], axis = 1)
title_df.head()

/tmp/ipykernel_30545/892722808.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_df = pd.read_csv(title_data, sep="\t")


,tconst,titleType,primaryTitle,startYear,genres
0,tt0000001,short,Carmencita,1894,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,1892,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,1892,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,1892,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,1893,"Comedy,Short"


In [45]:
title_df['startYear']= title_df['startYear'].replace("\\N","-1")
title_df['startYear'] = title_df['startYear'].astype("int32")
title_df['startYear']

13082      2021
61116      2020
76059      2022
102656     2021
117002     2022
           ... 
9639793    2020
9640153    2021
9640683    2020
9640722    2020
9640767    2020
Name: startYear, Length: 50521, dtype: int32

In [18]:
title_df = title_df[title_df['startYear']>=2020]
title_df = title_df[title_df['startYear']<=2022]
title_df = title_df[title_df['titleType'] == 'movie']
title_df = title_df[title_df['genres'] != "\\N"]
title_df

,tconst,titleType,primaryTitle,startYear,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,2021,Documentary
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,2020,Drama
76059,tt0077684,movie,Histórias de Combóios em Portugal,2022,Documentary
102656,tt0104988,movie,Neues in Wittstock,2021,Documentary
117002,tt0119830,movie,One Dog Day,2022,"Action,Comedy"
...,...,...,...,...,...
9639793,tt9914192,movie,No Gogó do Paulinho,2020,Comedy
9640153,tt9914972,movie,Blind Ambition,2021,Documentary
9640683,tt9916190,movie,Safeguard,2020,"Action,Adventure,Thriller"
9640722,tt9916270,movie,Il talento del calabrone,2020,Thriller


In [11]:
comp_pix_df = pix_df[pix_df['Interest'].str.contains('movie|film|hollywood',regex=True)]
comp_pix_df

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest
77,105056,STY1654458430,28407,USR1645868801,male,20,Chill Out with the First Poster for ‘I Am Groot’,The days are getting longer and the weather’s ...,https://image.pixstory.com/Pixstory-image-1654...,2022-02-26 15:16:41,"marvel superheroes, marvelcomics, hollywood se..."
80,105052,STY1654457806,11045,USR1637258628,others,22,Priya Dutt fondly remembers father Sunil Dutt ...,IN PRIYA DUTT’S OWN WORDS:\n\nDad was born on ...,https://image.pixstory.com/Pixstory-image-1654...,2021-11-18 23:33:48,"bollywood, bollywood music, bollywood style, b..."
84,105048,STY1654457339,11045,USR1637258628,others,22,Soon-to-be parents Sonam Kapoor and Anand Ahuj...,nam Kapoor and Anand Ahuja are on cloud nine a...,https://image.pixstory.com/Pixstory-image-1654...,2021-11-18 23:33:48,"bollywood, bollywood music, bollywood style, b..."
102,105029,STY1654450105,27909,USR1645083572,male,24,Official poster of “I am Groot” is here!!,Marvel Studios just announced the date of thei...,https://image.pixstory.com/Pixstory-image-1654...,2022-02-17 13:09:32,"i am groot, marvel, marvel superheroes, marvel..."
134,104997,STY1654441934,38594,USR1653203738,NaN,20,Best TV show couples: In my opinion,Jane Villanueva and Michael Cordero (Jane the ...,https://image.pixstory.com/Pixstory-image-1654...,2022-05-22 12:45:38,"shows, couples, netflix, love, friends, office..."
...,...,...,...,...,...,...,...,...,...,...,...
94679,131057,STY1661233444,29857,USR1648876553,NaN,48,Malayankunju - a review,Malayankunju’ is a good watch but it’s not a ...,https://image.pixstory.com/Pixstory-image-1661...,2022-04-02 10:45:53,"movies, books vs movie, reading, travel, writing"
94691,131045,STY1661230818,11447,USR1637905680,female,22,House of the Dragon’ Premiere Draws Nearly 10 ...,"<p dir=""ltr"">HBO's ""House of the Dragon"" premi...",https://image.pixstory.com/Pixstory-image-1661...,2021-11-26 11:18:00,"Technology, History, Food, Entertainment, Spor..."
94724,131012,STY1661224535,11045,USR1637258628,others,22,Hrithik Roshan Vs Ranbir Kapoor who is better ...,<i><b>WELL IT COMES TO TERMS OF ACTING LOOKS C...,https://image.pixstory.com/Pixstory-image-1661...,2021-11-18 23:33:48,"Entertainment, Environment, Inequality, Health..."
94909,130825,STY1661184122,71819,USR1660337948,female,12,Bollywood actresses who charged more fees than...,Gender-based pay disparity doesn’t only exist ...,https://image.pixstory.com/Pixstory-image-1661...,2022-08-13 02:29:08,"#actress, #bollywood celebrities, #charges, #b..."


In [15]:
rating_data = "data/title.ratings.tsv/data.tsv"
rating_df = pd.read_csv(rating_data, sep="\t")
rating_df

,tconst,averageRating,numVotes
0,tt0000001,5.7,1954
1,tt0000002,5.8,263
2,tt0000003,6.5,1787
3,tt0000004,5.6,179
4,tt0000005,6.2,2589
...,...,...,...
1282653,tt9916730,8.3,10
1282654,tt9916766,7.0,21
1282655,tt9916778,7.2,36
1282656,tt9916840,8.8,6


In [19]:
merged_df = pd.merge(title_df, rating_df, on='tconst', how='outer')
merged_df

,tconst,titleType,primaryTitle,startYear,genres,averageRating,numVotes
0,tt0013274,movie,Istoriya grazhdanskoy voyny,2021.0,Documentary,6.6,49.0
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,2020.0,Drama,6.4,166.0
2,tt0077684,movie,Histórias de Combóios em Portugal,2022.0,Documentary,NaN,NaN
3,tt0104988,movie,Neues in Wittstock,2021.0,Documentary,8.4,8.0
4,tt0119830,movie,One Dog Day,2022.0,"Action,Comedy",7.2,28.0
...,...,...,...,...,...,...,...
1308716,tt9916730,NaN,NaN,NaN,NaN,8.3,10.0
1308717,tt9916766,NaN,NaN,NaN,NaN,7.0,21.0
1308718,tt9916778,NaN,NaN,NaN,NaN,7.2,36.0
1308719,tt9916840,NaN,NaN,NaN,NaN,8.8,6.0


In [37]:
count = 0
for i in comp_pix_df.iterrows():
    for j in merged_df['primaryTitle']:
        if j in i[1]['Narrative']:
            count += 1
count

6609